## Final Project Submission

Please fill out:
* Student name: Claire Sarraille
* Student pace: self paced
* Scheduled project review date/time:  Monday June 14th 1pm PST
* Instructor name: Jeff Herman
* Project Repo URL: https://github.com/clairesarraille/mod1finproj/tree/main/dsc-mod-1-project-v2-1-online-ds-sp-000-master
* Blog post URL:


Set environment before running this script with: conda activate learn-env
Import datascience libraries:

In [212]:
# Import datascience libraries:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Import libraries for fetching and formatting data:
from tmdbv3api import TMDb, Discover, Movie
import os

# Import libraries for performing operations:
from math import ceil

# Load classes from tmdbv3api - a python library for The Movie Database (TMDb) API
tmdb = TMDb()
discover = Discover()
movie = Movie()

# Store my API key in my previously set environment variable as a tmdb.api_key string
priv_api_key = os.environ.get('TMDB_PRIVATE_API_KEY')
tmdb.api_key = priv_api_key

Preview of discover.discover_movies data:

I used this function to quickly scan a sub-set of titles & tweak variables such as:
- 'with_original_language': 'en'

I used a slightly different function to pretty-print samples of all info for a sub-set of movies. See appendix.

In [213]:


def print_sample_tmdb(m, m_r, p, y):
    print()
    print(f"Year is: {y}")
    print(f"Page is: {p +1}")
    print(f"There are {len(m[y])} movies in movies dict for {y}")
    for d in m_r[0:1]:
        print("sample original title: ",(dict(d))['original_title'])
    print()

Function to extract 500 movies each for years 2016-2021 inclusive, sorted by revenue descending
This line is commented out in the code below, to preserve space in the notebook:
- print_sample_tmdb(movies, movies_running, page, year)

In [215]:
# This function uses the discover.discover_movies() method from the tmdbv3api package to intake initial movie data:

# Define Constants
# Constants are usually defined on a module level and written in all capital letters with underscores separating words
N_YEAR_RANGE = range(2016, 2022)  # grab 2016-2021 data, inclusive
N_RESULT = 500  # Enter number of results desired for each year
N_RESULT_PAGE = 20  # Discover() class returns 20 results per page
# Num pages = Num results / num results per page, rounded UP using math.ceil
N_PAGES = ceil(N_RESULT / N_RESULT_PAGE)

# Instantiate empty dictionary movies to store outside of the following loops,
# because you want to use movies after loops are finished executing
movies = {}

# Function to grab data from discover.discover_movies:


def return_discover_movies():
    for year in N_YEAR_RANGE:  # iterate through years 2000-2019 inclusive
        # Keys are integers for each year
        # Values will be lists of tmdbv3api objects, converted to dictionaries
        movies[year] = []
        for page in range(0, N_PAGES):
            # For N_PAGES, create list movies_running, querying results by year, page, & sort revenue desc
            # The objects within movies_running 'tmdbv3api.as_obj.AsObj' objects, and will need to be converted to dictionaries later
            movies_running = discover.discover_movies({
                # We want to sort by revenue so that we make sure we're getting the highest grossing movies in each batch of 500
                'sort_by': 'revenue.desc', 
                'primary_release_year': str(year),
                # API is 1 indexed - so we grab pages 1 to N_PAGES inclusive
                'page': str(page + 1),
                'vote_count.gte': 1,    # Movies with 0 votes are likely among the most unpopular. OK to filter by at least 1 vote.
                'with_original_language': 'en' # Since our studio is still in R&D mode, we'll focus on movies originally shot in English
            })
            # For each year in our movies dictionary, store the year as a key (datatype list) in dictionary "movies"
            # Then, iterate through 'tmdbv3api.as_obj.AsObj' objects in movies_running for N_PAGES, converting each object as a dictionary
            # For each year we will have a list of dictionaries, and each dictionary is a set of information for a single movie
            # .extend used rather than .append in order to extend our list by >1 items at once
            movies[year].extend([dict(tmdb_obj)
                                for tmdb_obj in movies_running])
            
            # print_sample_tmdb(movies, movies_running, page, year)  # Preview original movie titles:


# Now we have our dictionary movies, with years 2000-2019 inclusive, each with 100 movies in each list from that year
# We didn't need to return movies in the definition of this function, because movies was declared before the function
return_discover_movies()

In [226]:
# Sanity check there are 3,000 titles in movies dictionary
# Print ALL movie titles and ids from movies dictionary to qa_titles.txt for data-cleaning prep: manual review
movie_count = 0
for year in movies:
    for moo_vie in range(0,len(movies[year])):
        movie_count +=1
        my_string = f"{movies[year][moo_vie]['title']}       {movies[year][moo_vie]['id']}"
        with open("/Users/clairesarraille/git-repos/mod1finproj/dsc-mod-1-project-v2-1-online-ds-sp-000-master/qa_files/qa_titles.txt", "a+",encoding='utf-8') as file_object:
            # Move read cursor to the start of file.
            file_object.seek(0)
            data = file_object.read(100)
            if len(data) > 0 :
                 file_object.write("\n")
            file_object.write(my_string)

# Print movie counts in movies dict:
print("The total movie count returned from our return_discover_movies() function above is: ")
print(movie_count)

# Print keys in movies dict:
print()
print("The keys in our movies dictionary are:")
print(movies.keys())


The total movie count returned from our return_discover_movies() function above is: 
3000

The keys in our movies dictionary are:
dict_keys([2016, 2017, 2018, 2019, 2020, 2021])


While scanning the above list, I found the following that I'll use to develop some data clean-up methods:

- These don't look legitimate:
    - movie_02    813075
    - movie_01    813064
    - welcome_home.exe    82418
    - jji    763071

- There's a double entry for this movie:
    - Split    358364
    - Split    381288

- Not in English, this one I later caught when running the analysis to find non-ascii characters:
    - عتيج    676624


In [223]:
# Check that API is working properly by spot-checking movie titles/ids between methods discover.discover_movies and movie.details()

# Expected data (from discover.movies)
# Bigfoot vs the Illuminati
# 784281
my_result = movie.details(784281)
print(my_result.title)
print(my_result.id)
print()

# Expected data (from discover.movies)
# Cheez-It's & Chili
# 802103
my_result = movie.details(802103)
print(my_result.title)
print(my_result.id)
print()

# Expected data (from discover.movies)
# A Tiny House Christmas
# 796438
my_result = movie.details(796438)
print(my_result.title)
print(my_result.id)
print()

Bigfoot vs the Illuminati
784281

Cheez-It's & Chili
802103

A Tiny House Christmas
796438



In [228]:
# Our movies dictionary has keys for years 2016-2021
# The value for each year is a list of dictionaries. Each dictionary is a movie.
# To turn this data into a Pandas datastructure, we iterate through all the years and all the dictionaries in each year
# Our result, movies_list_of_dict is a simple list of all the movie dictionaries.
# All we have to do then is use pd.DataFrame() to use the keys as column headers and the values as the cell values in rows.

movies_list_of_dict = [movie_d for movie_yr in movies for movie_d in movies[movie_yr]]
df_movies = pd.DataFrame(movies_list_of_dict)

In [252]:
# Preview df_movies:
# First three movies:
df_movies.head(3)

,title,backdrop_path
531,Get Out,/vZ7EVk7FaNEWYqlVWbFLHP8Z0LU.jpg


In [237]:
# Last three movies:
df_movies.tail(3)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
2997,False,/qtJGWUTaXuPVSqJOXSWoZCqucEr.jpg,"[35, 10749]",796126,en,Bad Cupid,Archie is a God on a mission to ensure that tr...,1.400,/9nSHoZpU8jFHjKwJ895bzkjPldG.jpg,2021-02-11,Bad Cupid,False,6.0,3
2998,False,None,"[80, 18, 10749]",796124,en,Ride or Die,"Ashley, a fierce friend to Mandy, takes the ph...",31.159,/77S8LQNkB0se2nk15X1d22QV1d1.jpg,2021-02-09,Ride or Die,False,4.0,2
2999,False,None,[18],796034,en,Oklahoma Mon Amour,Rural United States and modern urban Mexico me...,0.600,None,2021-04-24,Oklahoma Mon Amour,False,5.0,1


In [233]:
# See number of rows and columns, respectively:
df_movies.shape

(3000, 14)

In [244]:
# Are there any null values?
# It looks like the only null values are for backdrop_path (suffix for movie's image URL)
# In other words, no missing data for this analysis so far.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              3000 non-null   bool   
 1   backdrop_path      1931 non-null   object 
 2   genre_ids          3000 non-null   object 
 3   id                 3000 non-null   int64  
 4   original_language  3000 non-null   object 
 5   original_title     3000 non-null   object 
 6   overview           3000 non-null   object 
 7   popularity         3000 non-null   float64
 8   poster_path        2843 non-null   object 
 9   release_date       3000 non-null   object 
 10  title              3000 non-null   object 
 11  video              3000 non-null   bool   
 12  vote_average       3000 non-null   float64
 13  vote_count         3000 non-null   int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 287.2+ KB


In [254]:
type(df_movies['title'].str.contains('Get Out'))

pandas.core.series.Series

In [253]:
# Just for fun, let's explore one movie's backdrop_path - what do we do with this field?
# One of my favorite movies from the past 6 years is Get Out - so lets find it in df_movies
# I accessed a subset of df_movies with .loc using a pandas series of boolean values (title contains 'Get Out')
get_out = df_movies[['title', 'backdrop_path']].loc[(df_movies['title'].str.contains('Get Out'))]
get_out

,title,backdrop_path
531,Get Out,/vZ7EVk7FaNEWYqlVWbFLHP8Z0LU.jpg


Example back_drop path appended to https://image.tmdb.org/t/p/w500/ to get back the w500 size:
- https://image.tmdb.org/t/p/w500/vZ7EVk7FaNEWYqlVWbFLHP8Z0LU.jpg


- I embedded it here using <img src="get_out.png"\>, and storing the actual image get_out.png in the folder with this notebook:

<img src="get_out.png">

In [235]:
# Next, we'll create a list of movie ids we have stored in df_movies
# We'll use that list to get more detailed movie info from the tmdbv3api library's movie.details() method

# Create list of movie ids from df_movies:
movie_id_list = df_movies['id'].tolist()

# Check length of list:
len(movie_id_list)

3000

In [255]:
# Find any movies that would throw an error when passed to movie.details()
# Under the hood of the tmdbv3api library we are simply using the GET method to retrieve data from the TMDb server at a specified resource.
# The TMDb API behaves such that if the movie id doesn't exist in the /movie/{movie_id} resource, we'll get an error back
# Let's see if any of our movie ids would throw such an error:

my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

details_list = []

for id_index in movie_id_list:
    try:
        r = movie.details(id_index)
        details_list.extend({my_key: dict(r).get(my_key) for my_key in my_keys})
    except:
        print(id_index)
        continue

781739


In [258]:
# Out of curiosity, lets look at the details for this movie:
df_movies[['title', 'id', 'popularity', 'overview', 'release_date']].loc[(df_movies['id'] == 781739)]

,title,id,popularity,overview,release_date
2396,Hero,781739,0.0,Hero was diagnosed with cancer at the age of 2...,2020-12-31


In [262]:
print(df_movies.loc[(df_movies['id'] == 781739), 'overview'].item())

Hero was diagnosed with cancer at the age of 22. After losing faith in himself, and getting ready to give up, he found support in a place he felt safe - The Furry Fandom. Now, Hero is getting his first fursuit. This is his story.


In [ ]:
# There is one movie that threw an error. Check it out in our df_movies and then remove from movie_id_list
movie_id_list.remove(781739)


In [ ]:
my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

details_list = []

for id_index in movie_id_list:
    try:
        r = movie.details(id_index)
        details_list.extend({my_key: dict(r).get(my_key) for my_key in my_keys})
    except:
        print(id_index)
        continue

In [ ]:
# small_list_ids = [271110, 330459, 127380, 269149, 278927, 328111, 209112, 259316, 293660, 297761]

my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

# In case any of the movies are missing any of the above keys, use:
# dict.get(key, default_value)
# This is one way in which I dealt with missing values that I was receiving in the API

details_list = []
details_list.extend({my_key: dict(movie.details(id_index)).get(my_key) for my_key in my_keys} for id_index in movie_id_list)
print(details_list[0])

In [ ]:
# Convert details_list to a pandas df:
df_details = pd.DataFrame(details_list)
df_details[0:3]

In [ ]:
# Take initial stock:
df_details.index
df_details.info()
df_details.columns
df_details.dtypes


In [ ]:
# Future: Could check original language by connecting with IMDb & checking language from there - could also use LetterBox
# Detect non-english strings in title and remove them from df_details:

pd.set_option('display.max_rows', df_details.shape[0]+1) # Set display option so that we can see all rows because we will likely manually review the values:

# Create function to detect non-english strings:
def detect_en(ascii_string):
    try:
        ascii_string.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

column_names = ["id", "title"]
flag_non_en = pd.DataFrame(columns = column_names)
for item in df_details.loc[:, 'title']:
    if isEnglish(item) == False:
        flag_non_en = flag_non_en.append(df_details[['id','title']].loc[df_details['title'] == item])
# print(flag_non_en)


In [ ]:
# Create list of movie ids that you want to remove from df_details, based on internet research to see if original language is English:
# 720980      Express Yourself: Die Tänzer der Queen of Pop
# 676624                                               عتيج
# 671272                              Óleos en la oscuridad
# 664648                                          Crisálida
# 692856                                              Avīci
# 691836                      Avante (Ao Vivo em São Paulo)
# 680817                                      Hranice práce
# 676912                                            Sátiros
# 703348                                               عقلي
# 720570                                   maníaco da ponte
# 832125                                  Ghostemane: Ímpio
# 806831                 História Secreta do Pop Brasileiro
# 783999        Crimes de haine en Amérique : l'inquiétante
# 740354                                          Bach-Hông
# 793311                                   Toilette d’amour
# 787619                                           Czyściec
# 832310                                           Verrückt
# 822161                                         ¿Libertad?
# 802434                                              Acéré


# Remove rows with these ids from df_details:
non_eng_ids = [720980, 676624, 671272, 664648, 692856, 691836, 680817, 676912, 703348, 720570, 832125, 806831, 783999, 740354, 793311, 787619, 832310, 822161, 802434]


In [ ]:
# Preview the object columns and convert each variable to a separate Boolean column
# Make Keywords column one long string
my_sample = df_details.loc[0:5, 'genres']
type(my_sample)
my_sample
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)
# df_new = pd.json_normalize(df_details, 'genres', ['id','title'])


In [ ]:
def convert_to_list_of_dict(val):
    val = [dict(x) for x in val]
    return val

#   movies[year].extend([dict(tmdb_obj) for tmdb_obj in movies_running])
            # Preview original movie titles:
# def convert_to_list_of_dict(val):
#     list_dict_new = []
#     for val_obj_tmdb in val:
#         list_dict_new.append(dict(val_obj_tmdb))
#         return list_dict_new


df_details['genres_list_dict'] = df_details.loc[:,'genres'].apply(convert_to_list_of_dict)

my_sample = df_details.loc[0:5, 'genres_list_dict']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)

In [ ]:
df_details[0:3]

In [ ]:
# This normalization method doesn't quite work:
df_norm_details = pd.DataFrame(df_details['genres_list_dict'].tolist())
df_norm_details.head()

In [ ]:
old_cat_list = df_details['genres_list_dict'].tolist()

In [ ]:
old_cat_list[0:10]
# First list is for movie id 271110 -- check that those genre values are correct

In [ ]:
# Drop adult column:
df.drop('DESC', axis=1)

In [ ]:
# Re-order columns

In [ ]:
# Decide what to do with these
# These don't look legitimate:
movie_02
813075

movie_01
813064

welcome_home.exe
82418

jji
763071

# There's a double entry for this movie:
Split
358364
Split
381288